### Nathan Eskelson

In [85]:
import numpy as np
from scipy.stats import binom

## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

def EuropeanCallDelta(S, X, u, d):
    deltaC = (max((u * S) - X, 0) - max((d * S) - X, 0))/(S * (u - d))
    return deltaC

def EuropeanCallBeta(S, X, u, d, r, T, n):
    betaC = np.exp(-r * (T/n)) * (((u * (max((d * S) - X, 0))) - d * (max((u * S) - X, 0))) / (u - d))
    return betaC

def EuropeanCall(S, X, r, u, d, T, n):
    deltaC = EuropeanCallDelta(S, X, u, d) 
    betaC = EuropeanCallBeta(S, X, u, d, r, T, n)
    callPrice = S * deltaC + betaC
    return callPrice

def EuropeanPutDelta(S, X, u, d):
    deltaP = (max(X - (u * S), 0) - max(X - (d * S), 0))/(S * (u - d))
    return deltaP

def EuropeanPutBeta(S, X, u, d, r, T, n):
    betaP = np.exp(-r * (T/n)) * (((u * (max(X - (d * S), 0))) - d * (max(X - (u * S), 0))) / (u - d))
    return betaP

def EuropeanPut(S, X, r, u, d, T, n):
    deltaP = EuropeanPutDelta(S, X, u, d) 
    betaP = EuropeanPutBeta(S, X, u, d, r, T, n)
    putPrice = S * deltaP + betaP
    return putPrice

def buildTrees(S,X,r,T,v,u,d,n,opt_type):
    spotArr = np.zeros([n + 1, n + 1])
    payoffArr = np.zeros([n + 1, n + 1])
    premArr = np.zeros([n + 1, n + 1])
    deltaArr = np.zeros([n + 1, n + 1])
    betaArr = np.zeros([n + 1, n + 1])
    # build price tree
    for x in range(n + 1):
        for y in range(n + 1):
            if (y - x) >= 0:
                spotArr[x,y] = S * (u ** (y - x)) * (d ** (x))
    # build payoff tree with prem, delta, and beta
    for x in range(n + 1):
        for y in range(n + 1):
            if (y - x) >= 0:
                payoffArr[x,y] = max(spotArr[x,y] - X, 0)
                if opt_type == 'call':
                    premArr[x,y] = EuropeanCall(spotArr[x,y], X, r, u, d, T, n)
                    deltaArr[x,y] = EuropeanCallDelta(spotArr[x,y], X, u, d)
                    betaArr[x,y] = EuropeanCallBeta(spotArr[x,y], X, u, d, r, T, n)
                else:
                    premArr[x,y] = EuropeanPut(spotArr[x,y], X, r, u, d, T, n)
                    deltaArr[x,y] = EuropeanPutDelta(spotArr[x,y], X, u, d)
                    betaArr[x,y] = EuropeanPutBeta(spotArr[x,y], X, u, d, r, T, n)
    
    optionValues = {'price':spotArr, 'payoff':payoffArr, 'premium':premArr,'delta':deltaArr, 'beta':betaArr}
    return optionValues

def americanBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-r * h)
    
    ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
        
    if verbose:
        print(Ct)
        
    for t in range((n-1), -1, -1):
        for j in range(t + 1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            St[j] = St[j] / u
            np.maximum(Ct[j], payoff(St[j], K))
            print(Ct)
            
    return Ct[0]

def CallPayOff(Spot, Strike):
    return np.maximum(Spot - Strike, 0.0)

def EuropeanBinomial(S, X, r, u, d, T, n):
    numSteps = n
    numNodes = numSteps + 1
    spotT = 0.0
    callT = 0.0
    pu = (np.exp(r*T) - d) / (u - d)
    pd = 1 - pu
    
    for i in range(numNodes):
        spotT = S * (u ** (numSteps - i)) * (d ** (i))
        callT += CallPayOff(spotT, X) * binom.pmf(numSteps - i, numSteps, pu)  
    callPrice = callT * np.exp(-r * T)
     
    return callPrice
    
buildTrees(100, 95, .08, .5, 0, 1.3, .8, 2, 'call')

{'price': array([[100., 130., 169.],
        [  0.,  80., 104.],
        [  0.,   0.,  64.]]), 'payoff': array([[ 5., 35., 74.],
        [ 0.,  0.,  9.],
        [ 0.,  0.,  0.]]), 'premium': array([[15.10887429, 36.88112604, 75.88112604],
        [ 0.        ,  3.8851391 , 17.35362133],
        [ 0.        ,  0.        ,  0.        ]]), 'delta': array([[0.7       , 1.        , 1.        ],
        [0.        , 0.225     , 0.77307692],
        [0.        , 0.        , 0.        ]]), 'beta': array([[-54.89112571, -93.11887396, -93.11887396],
        [  0.        , -14.1148609 , -63.04637867],
        [  0.        ,   0.        ,   0.        ]])}

## 1)

In [10]:
def main_1():
    S = 100
    X = 105
    r = 0.08
    T = 0.5
    v = 0.0
    u = 1.3
    d = 0.8
    n = 1
    
    callPrice = EuropeanCall(S, X, r, u, d, T, n)
    print("Call Premium = {0:.4f}".format(callPrice))
    callDelta = EuropeanCallDelta(S, X, u, d)
    print("Call Delta = {0:.4f}".format(callDelta))
    callBeta = EuropeanCallBeta(S, u, d, r, T, n, X)
    print("Call Beta = {0:.4f}".format(callBeta))
    putPrice = EuropeanPut(S, X, r, u, d, T, n)
    print("Put Premium = {0:.4f}".format(putPrice))
    putDelta = EuropeanPutDelta(S, X, u, d)
    print("Put Delta = {0:.4f}".format(putDelta))
    putBeta = EuropeanPutBeta(S, u, d, r, T, n, X)
    print("Put Beta = {0:.4f}".format(putBeta))
    
main_1()

Call Premium = 11.5684
Call Delta = 0.5000
Call Beta = -1.2938
Put Premium = 12.4513
Put Delta = -0.5000
Put Beta = 0.0000


## 2)

In [11]:
def main_2():
    S = 100
    X = 95
    r = 0.08
    T = 0.5
    v = 0.0
    u = 1.3
    d = 0.8
    n = 1
    
    putPrice = EuropeanPut(S, X, r, u, d, T, n)
    print("a. Put Price = {0:.4f}".format(putPrice))
    callPrice = EuropeanCall(S, X, r, u, d, T, n)
    print("b. Call Price = {0:.4f}".format(callPrice)) 
    print("\n Buy 17 shares of the stock and borrow 53.8042 to create a synthetic call. After we will sell the call option and make 17 - 16.1958 which is .8042\n")
    print("c. We want to buy the call option as they have it undervalued. After sell 17 units of a share and lend 53.8042. We then profit 16.1958 - 15 which is 1.1958")
    
main_2()

a. Put Price = 7.4708
b. Call Price = 16.1958

 Buy 17 shares of the stock and borrow 53.8042 to create a synthetic call. After we will sell the call option and make 17 - 16.1958 which is .8042

c. We want to buy the call option as they have it undervalued. After sell 17 units of a share and lend 53.8042. We then profit 16.1958 - 15 which is 1.1958


## 3)

In [30]:
def main_3():
    S = 100
    X = 95
    r = 0.08
    T = 0.5
    v = 0.00
    u = 1.3
    d = 0.8
    n = 2
    
    trees3 = buildTrees(S,X,r,T,v,u,d,n,opt_type = 'call') 
    for key, value in trees3.items():
        print(key,'\n',value,'\n')
main_3()    

price 
 [[100. 130. 169.]
 [  0.  80. 104.]
 [  0.   0.  64.]] 

payoff 
 [[ 5. 35. 74.]
 [ 0.  0.  9.]
 [ 0.  0.  0.]] 

premium 
 [[15.10887429 36.88112604 75.88112604]
 [ 0.          3.8851391  17.35362133]
 [ 0.          0.          0.        ]] 

delta 
 [[0.7        1.         1.        ]
 [0.         0.225      0.77307692]
 [0.         0.         0.        ]] 

beta 
 [[-54.89112571 -93.11887396 -93.11887396]
 [  0.         -14.1148609  -63.04637867]
 [  0.           0.           0.        ]] 



## 4)

In [13]:
def main_4():
    S = [80, 90, 110, 120, 130]
    X = 95
    r = 0.08
    T = 0.5
    v = 0.00
    u = 1.3
    d = 0.8
    n = 2
    
    for i in S:
        print('Delta Trees for Spot price of ${0:.0f}'.format(i))
        trees4 = buildTrees(i,X,r,T,v,u,d,n,opt_type = 'call')
        print(trees4['delta'],'\n')
main_4()    

Delta Trees for Spot price of $80
[[0.225      0.77307692 1.        ]
 [0.         0.         0.31634615]
 [0.         0.         0.        ]] 

Delta Trees for Spot price of $90
[[0.48888889 0.97606838 1.        ]
 [0.         0.         0.57008547]
 [0.         0.         0.        ]] 

Delta Trees for Spot price of $110
[[0.87272727 1.         1.        ]
 [0.         0.44090909 0.93916084]
 [0.         0.         0.        ]] 

Delta Trees for Spot price of $120
[[1.         1.         1.        ]
 [0.         0.62083333 1.        ]
 [0.         0.         0.12604167]] 

Delta Trees for Spot price of $130
[[1.         1.         1.        ]
 [0.         0.77307692 1.        ]
 [0.         0.         0.31634615]] 



## 5a)

In [52]:
S = 100
X = 95.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
n = 3

callPrc = americanBinomPricerRecursive(S, X, r, v, q, T, n, callPayoff, verbose = True)
print(f"The American Call Premium: {callPrc : 0.3f}")

[87.1417861 33.8147424  0.         0.       ]
[56.64406241 33.8147424   0.          0.        ]
[56.64406241 15.04032855  0.          0.        ]
[56.64406241 15.04032855  0.          0.        ]
[33.14931753 15.04032855  0.          0.        ]
[33.14931753  6.6897296   0.          0.        ]
[18.28255221  6.6897296   0.          0.        ]
The American Call Premium:  18.283


## 5b)

In [88]:
def main():
    S = 100
    X = 95
    r = 0.08
    T = 1
    n = 3
    v = 0.3
    q = 0
    h = T / n
    u = np.exp((r - v) * h + q * np.sqrt(h))
    d = np.exp((r - v) * h - q * np.sqrt(h))
    
    callPrice = EuropeanBinomial(S, X, r, u, d, T, n)
    print("The European Call Price is = {0:.4f}".format(callPrice))
                                                
main()

The European Call Price is = nan


C:\Users\neske\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in double_scalars


## 5c)

In [90]:
S = 100
X = 95.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
n = 3

putPrc = americanBinomPricerRecursive(S, X, r, v, q, T, n, putPayoff, verbose = True)
print(f"The European Put Premium: {putPrc : 0.3f}")

[ 0.          0.          3.89933488 30.57157332]
[ 0.          0.          3.89933488 30.57157332]
[ 0.          2.06235672  3.89933488 30.57157332]
[ 0.          2.06235672 17.90366345 30.57157332]
[ 1.09077967  2.06235672 17.90366345 30.57157332]
[ 1.09077967 10.38654838 17.90366345 30.57157332]
[ 5.97860511 10.38654838 17.90366345 30.57157332]
The European Put Premium:  5.979


## 5d)

In [51]:
S = 100
X = 95.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
n = 3

putPrc = americanBinomPricerRecursive(S, X, r, v, q, T, n, putPayoff, verbose = True)
print(f"The American Put Premium: {putPrc : 0.3f}")

[ 0.          0.          3.89933488 30.57157332]
[ 0.          0.          3.89933488 30.57157332]
[ 0.          2.06235672  3.89933488 30.57157332]
[ 0.          2.06235672 17.90366345 30.57157332]
[ 1.09077967  2.06235672 17.90366345 30.57157332]
[ 1.09077967 10.38654838 17.90366345 30.57157332]
[ 5.97860511 10.38654838 17.90366345 30.57157332]
The American Put Premium:  5.979


## 6a)

In [28]:
def q_6():
    S = 40
    X = 40
    r = 0.08
    T = 0.5
    v = 0.00
    q = 0.30
    n = 3
    h = T / n
    u = np.exp((r - v) * h + q * np.sqrt(h))
    d = np.exp((r - v) * h - q * np.sqrt(h))
    
    print("u = {0:.4f}".format(u))
    print("d = {0:.4f}".format(d))
    
q_6()

u = 1.1455
d = 0.8966


## 6b)